코드 4-1

In [1]:
!pip install ultralytics opencv-python torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.4 MB/s eta 0:00:00


In [3]:
import cv2
from ultralytics import YOLO
from PIL import Image

# 1. 모델 로딩
model = YOLO("yolov8n.pt")

# 2. 이미지 불러오기
img_path = "person.png"
image = cv2.imread(img_path)

# 3. YOLO 탐지 실행
results = model.predict(source=image, conf=0.5, verbose=False)[0]

# 4. 탐지된 객체 중 'person' 클래스만 crop
target_class = "person"
for i, box in enumerate(results.boxes):
    cls_id = int(box.cls[0].item())
    cls_name = model.names[cls_id]

    if cls_name == target_class:
        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
        cropped = image[y1:y2, x1:x2]

        # 5. 잘라낸 이미지 저장
        save_path = f"cropped_{target_class}_{i}.jpg"
        cv2.imwrite(save_path, cropped)
        print(f"📁 저장 완료: {save_path}")


WARNING ⚠️ 'source' is missing. Using 'source=/usr/local/lib/python3.12/dist-packages/ultralytics/assets'.


TypeError: 'NoneType' object is not subscriptable

코드 4-2

In [ ]:
import cv2
from ultralytics import YOLO

# 1. 모델 로딩 및 이미지 불러오기
model = YOLO("yolov8n.pt")
image = cv2.imread("/content/group.png")
result = model.predict(source=image, conf=0.5, verbose=False)[0]

# 2. 대체 이미지 로드 (예: 이모지, 마스킹 아이콘)
replacement_img = cv2.imread("/content/imoticon.png")  # 반드시 배경 투명 제외하고 RGB 이미지로 준비

# 3. 탐지 결과 후처리
for box in result.boxes:
    cls_id = int(box.cls[0].item())
    cls_name = model.names[cls_id]
    x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())

    roi = image[y1:y2, x1:x2]

    # 블러 처리
    # blurred = cv2.GaussianBlur(roi, (51, 51), 0)
    # image[y1:y2, x1:x2] = blurred

    # 색상 마스킹 (빨간 박스 채우기)
    # cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), -1)

    # 대체 이미지 삽입
    resized_icon = cv2.resize(replacement_img, (x2 - x1, y2 - y1))
    image[y1:y2, x1:x2] = resized_icon

# 4. 결과 저장
cv2.imwrite("output_processed.jpg", image)
print("💾 결과 이미지 저장 완료: output_processed.jpg")


💾 결과 이미지 저장 완료: output_processed.jpg


코드 4-3

In [4]:
!pip install  pyngrok -q

🔹 1. 토큰 발급
https://dashboard.ngrok.com/signup 에서 가입 → 이메일 인증

https://dashboard.ngrok.com/get-started/your-authtoken 에서 Authtoken 복사

🔹 2. Postman 설치 및 실행

Postman 다운로드 : https://www.postman.com/downloads/

설치 후 실행

🔹 3. Postman에서 접속
위 Public URL을 복사해서 Postman에 붙여넣으세요.

POST 요청:

http://xxxx.ngrok-free.app/echo

Body → raw → JSON:

{ "msg": "hello from postman" }

In [6]:
NGROK_TOKEN = "31oYGjMw3hmoZ2rPifbe1KUpFr9_KhhsxCokQm3vWHnkHNim"

from pyngrok import ngrok, conf
conf.get_default().auth_token = NGROK_TOKEN

# Flask + ngrok
from flask import Flask, request, jsonify
from pyngrok import ngrok

app = Flask(__name__)

@app.route("/")
def home():
    return "Hello from Flask (Colab + ngrok)!"

@app.route("/echo", methods=["POST"])
def echo():
    return jsonify({"you_sent": request.get_json()})

# 5000 포트로 Flask 실행 & ngrok 터널 오픈
public_url = ngrok.connect(5000)
print("🌐 Public URL:", public_url)

app.run(port=5000)

🌐 Public URL: NgrokTunnel: "https://7ff2a93735b3.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [26/Aug/2025 07:22:56] "POST /echo HTTP/1.1" 415 -
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
a

코드 4-4 (원본)

In [ ]:
from flask import Flask, request, send_file
from ultralytics import YOLO
import cv2
import numpy as np
import io

app = Flask(__name__)
model = YOLO("yolov8n.pt")  # 사전학습된 YOLOv8 모델 로드

@app.route('/detect', methods=['POST'])
def detect_yolo():
    if 'image' not in request.files:
        return {'error': 'No image file provided'}, 400

    file = request.files['image']
    image_data = np.frombuffer(file.read(), np.uint8)
    image = cv2.imdecode(image_data, cv2.IMREAD_COLOR)

    # YOLO 탐지 실행
    results = model.predict(source=image, conf=0.5, verbose=False)[0]

    # 탐지된 객체에 박스 그리기
    for box in results.boxes:
        cls_id = int(box.cls[0])
        label = model.names[cls_id]
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    # 이미지 인코딩 후 반환
    _, buffer = cv2.imencode('.jpg', image)
    return send_file(
        io.BytesIO(buffer.tobytes()),
        mimetype='image/jpeg',
        as_attachment=False,
        download_name='result.jpg'
    )

if __name__ == '__main__':
    app.run(debug=True, port=5000)


코드 4-4 (ngrok)

### Postman 호출 단계
URL 설정: 먼저, 파이썬 코드를 실행했을 때 출력되는 ngrok의 공개 URL을 복사하세요. 이 URL 끝에 /detect를 붙여서 요청할 URL을 완성합니다.

POST 메서드를 선택하고, URL 입력란에 다음과 같이 입력하세요.

https://[복사한-ngrok-주소].ngrok-free.app/detect

데이터 설정: Postman에서 이미지를 첨부해야 합니다.

Body 탭을 클릭하세요.

form-data를 선택하세요.

파일 첨부: form-data의 키(Key)와 값(Value)을 설정합니다.

**Key: image 라고 입력하세요. (Flask 코드에서 request.files에 접근하는 키 이름과 일치해야 합니다.)**

**Value: Key 입력란 오른쪽에 있는 드롭다운 메뉴를 Text에서 File로 변경하세요. =(ex. cat.png) **

Select File 버튼을 클릭해서 YOLO로 감지하고 싶은 이미지 파일을 선택하세요.

요청 전송: Send 버튼을 눌러 요청을 보내면 됩니다.



In [ ]:
NGROK_TOKEN = "31oYGjMw3hmoZ2rPifbe1KUpFr9_KhhsxCokQm3vWHnkHNim" # Reuse the ngrok token
from pyngrok import ngrok, conf
conf.get_default().auth_token = NGROK_TOKEN

# Flask + ngrok setup
from flask import Flask, request, send_file
from ultralytics import YOLO
import cv2
import numpy as np
import io

app = Flask(__name__)
model = YOLO("yolov8n.pt")  # Load the pre-trained YOLOv8 model

@app.route('/detect', methods=['POST'])
def detect_yolo():
    if 'image' not in request.files:
        return {'error': 'No image file provided'}, 400

    file = request.files['image']
    image_data = np.frombuffer(file.read(), np.uint8)
    image = cv2.imdecode(image_data, cv2.IMREAD_COLOR)

    # Run YOLO detection
    results = model.predict(source=image, conf=0.5, verbose=False)[0]

    # Draw bounding boxes on the detected objects
    for box in results.boxes:
        cls_id = int(box.cls[0])
        label = model.names[cls_id]
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    # Encode the image and return
    _, buffer = cv2.imencode('.jpg', image)
    return send_file(
        io.BytesIO(buffer.tobytes()),
        mimetype='image/jpeg',
        as_attachment=False,
        download_name='result.jpg'
    )

if __name__ == '__main__':
    # Open ngrok tunnel and run Flask app
    public_url = ngrok.connect(5000)
    print("🌐 Public URL:", public_url)
    app.run(debug=False, port=5000)

🌐 Public URL: NgrokTunnel: "https://21ad60131f42.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information abo